# Lab 4

In [1]:
import numpy as np
import pandas as pd 
import seaborn as sns 
import matplotlib as mp
import matplotlib.pyplot as plt 

from scipy import stats
from textwrap import wrap

pd.set_option('precision', 10)

%matplotlib inline
%config InlineBackend.figure_format = 'pdf'

In [2]:
CL = pd.read_csv('/Users/kev/Downloads/BCMB 301A Lab Techniques/Lab 4 - Partial Purification of beta-galactosidase from E. coli CSH66 Cell-Free Lysate/Data/Crude Lysate.csv',
                 header=1)

SN = pd.read_csv('/Users/kev/Downloads/BCMB 301A Lab Techniques/Lab 4 - Partial Purification of beta-galactosidase from E. coli CSH66 Cell-Free Lysate/Data/Subernatent.csv', 
                 header=1)

SE = pd.read_csv('/Users/kev/Downloads/BCMB 301A Lab Techniques/Lab 4 - Partial Purification of beta-galactosidase from E. coli CSH66 Cell-Free Lysate/Data/SEC.csv',
                 header=1)

DE = pd.read_csv('/Users/kev/Downloads/BCMB 301A Lab Techniques/Lab 4 - Partial Purification of beta-galactosidase from E. coli CSH66 Cell-Free Lysate/Data/DEAE.csv', 
                 header=1)

BF = pd.read_csv('/Users/kev/Downloads/BCMB 301A Lab Techniques/Lab 4 - Partial Purification of beta-galactosidase from E. coli CSH66 Cell-Free Lysate/Data/Bradford.csv', 
                 header=1)

MW = pd.read_csv('/Users/kev/Downloads/BCMB 301A Lab Techniques/Lab 4 - Partial Purification of beta-galactosidase from E. coli CSH66 Cell-Free Lysate/Data/MWvDist.csv', 
                 header=1)

In [3]:
CL_ave = pd.DataFrame(CL.iloc[:,1:].mean(axis=1))
CL_ave.columns = ['CL Ave']
CL = CL.join(CL_ave)

SN_ave = pd.DataFrame(SN.iloc[:,1:].mean(axis=1))
SN_ave.columns = ['CN Ave']
SN = SN.join(SN_ave)

## Absorbances

In [4]:
Ave = pd.concat([CL_ave, SN_ave, SE.iloc[:,1], DE.iloc[:,1]], sort=False, ignore_index=True, axis=1)
Ave.columns = ['Lysate', 'Supernatent', 'SEC', 'DEAE']
Ave

,Lysate,Supernatent,SEC,DEAE
0,0.0000,0.0000,0.000,0.000
1,0.0690,0.0105,0.372,0.388
2,0.1375,0.0210,0.557,0.626
3,0.2055,0.0325,0.636,0.742
4,0.2760,0.0425,0.660,0.794
5,0.3440,0.0535,0.671,0.811
6,0.4255,0.0645,0.673,0.819
7,0.4845,0.0755,0.675,0.821
8,0.5585,0.0880,0.677,0.825
9,0.6330,0.0995,0.679,0.824


In [5]:
Diff = Ave.diff()
Diff = Diff.loc[1:,:]
Diff

,Lysate,Supernatent,SEC,DEAE
1,0.0690,0.0105,0.372,0.388
2,0.0685,0.0105,0.185,0.238
3,0.0680,0.0115,0.079,0.116
4,0.0705,0.0100,0.024,0.052
5,0.0680,0.0110,0.011,0.017
6,0.0815,0.0110,0.002,0.008
7,0.0590,0.0110,0.002,0.002
8,0.0740,0.0125,0.002,0.004
9,0.0745,0.0115,0.002,-0.001
10,0.0685,0.0120,0.000,0.000


In [6]:
Diff_mean = [Diff.loc[:,'Lysate'].mean(axis=0), Diff.loc[:,'Supernatent'].mean(axis=0),
             Diff.loc[0:8,'SEC'].mean(axis=0), Diff.loc[0:7,'DEAE'].mean(axis=0)]
Diff_mean = pd.DataFrame(Diff_mean).T
Diff_mean.columns = ['Lysate', 'Supernatent', 'SEC', 'DEAE']
Diff_mean

,Lysate,Supernatent,SEC,DEAE
0,0.0692083333,0.011125,0.084625,0.1172857143


## Standard curve for BSA

In [7]:
BS = BF.iloc[0:7,:].astype('float64')
BS.columns = [r'BSA ($\mu$l)', 'Set 1', 'Set 2']
BS_ave = pd.DataFrame(BS.iloc[:,1:].mean(axis=1))
BS_m = BS[r'BSA ($\mu$l)'] * 0.1
BS = pd.concat([BS, BS_ave, BS_m], ignore_index=True, sort=False, axis = 1)
BS.columns = [r'BSA ($\mu$l)', 'Set 1', 'Set 2', 'Ave', r'BSA ($\mu$g)']
BS

,BSA ($\mu$l),Set 1,Set 2,Ave,BSA ($\mu$g)
0,0.0,-0.003,0.003,0.0000,0.0
1,40.0,0.060,0.063,0.0615,4.0
2,80.0,0.114,0.120,0.1170,8.0
3,120.0,0.167,0.171,0.1690,12.0
4,200.0,0.253,0.303,0.2780,20.0
5,250.0,0.392,0.359,0.3755,25.0
6,300.0,0.457,0.456,0.4565,30.0


In [8]:
def plot_abs(Dat, X, Y, Abs, Yerr, ploterror=False):
    slope, intercept, r_value, p_value, std_err = stats.linregress(Dat[X], Dat[Y])

    plt.figure()
    sns.regplot(x = X, y=Y, data=Dat, 
                scatter_kws={'s':8}, 
                line_kws={'linewidth':2, 'label':r"A={0:.3g}M+({1:.3g}); R$^2$={2:.3g}".format(slope, intercept, r_value**2)})
    
    if ploterror == True: 
        plt.errorbar(x=Dat[X], y=Dat[Y], yerr=Yerr, scatterpoints=None, linewidth=0, elinewidth=1, capsize=2,
                    label='Uncertainty: {0:.4f}'.format(Yerr))
    
#     F_title = r'Changes in average absorbance at {0} nm ($\bar{{A}}_{1}$) with respect to the quantity of BSA, with a 95% confidence interval'.format(Abs, {Abs})
#     plt.title("\n".join(wrap(F_title, 80)))
    plt.ylabel(r'$\bar{{A}}_{0}$'.format({Abs}))
    plt.legend(loc=2)
    plt.grid()
    plt.show()
    return slope, intercept

In [9]:
BSs, BSi = plot_abs(Dat=BS, X=r'BSA ($\mu$g)', Y='Ave', Abs=595, Yerr=0)

<Figure size 432x288 with 1 Axes>

In [10]:
BP = BF.iloc[8:,:]
BP_ave = pd.DataFrame(BP.iloc[:, 1:].mean(axis=1))
BP_m = (BP_ave - BSi)/BSs
BP = pd.concat([BP, BP_ave, BP_m], axis=1, ignore_index=False, sort=False)
BP.columns = ['Sample', 'Set 1', 'Set 2', 'Ave', r'BSA ($\mu$l)']
BP

,Sample,Set 1,Set 2,Ave,BSA ($\mu$l)
8,Crude,0.125,0.118,0.1215,8.3652113124
9,Supernatant,-0.006,-0.007,-0.0065,-0.1632411821


## Enzyme table

In [11]:
Vol = [0.98, 3.4 , 2.5 , 0.9]
Dil = [2000, 20, 400, 200]

Volu = pd.DataFrame(Vol).T
Volu.columns = list(Diff_mean.columns)
Conc = pd.DataFrame(Diff_mean * 6 * Dil / 3 )
EnzT = pd.DataFrame(Conc * Vol)
ProC = pd.DataFrame((Diff_mean - BSi)/BSs)
ProT = pd.DataFrame(ProC * Vol)
SpAc = pd.DataFrame(EnzT / ProT )
ProR = pd.DataFrame(ProT / float(ProT['Lysate']) * 100)
EnzR = pd.DataFrame(EnzT / float(EnzT['Lysate']) * 100)
FdPR = pd.DataFrame(SpAc / float(SpAc['Lysate']))

EnzTab = pd.concat([Volu, Conc, EnzT, ProC, ProT, SpAc, ProR, EnzR, FdPR], axis=0, sort=False, ignore_index=True)

In [12]:
EnzTab

,Lysate,Supernatent,SEC,DEAE
0,0.9800000000,3.4000000000,2.5000000000,0.9000000000
1,276.8333333333,0.4450000000,67.7000000000,46.9142857143
2,271.2966666667,1.5130000000,169.2500000000,42.2228571429
3,4.8810941638,1.0110867493,5.9082840801,8.0844196287
4,4.7834722805,3.4376949476,14.7707102002,7.2759776658
5,56.7154256906,0.4401204944,11.4584876222,5.8030493058
6,100.0000000000,71.8660994775,308.7863655125,152.1066129200
7,100.0000000000,0.5576920714,62.3855803610,15.5633527170
8,1.0000000000,0.0077601550,0.2020347636,0.1023187120


In [13]:
EnzTab.rename(index={0:'Volume (ml)', 
                     1:'Enzyme activity (units/ml)',
                     2:'Total enzyme (units)',
                     3:'[Protein] (mg/ml)', 
                     4:'Total protein (mg)',
                     5:'Specific activity (units/mg)',
                     6:'Total protein relative to lysate (%)',
                     7:'Enzyme yield relative to lysate (%)',
                     8:'Fold purification relative to lysate'}, inplace=True)

EnzTab

,Lysate,Supernatent,SEC,DEAE
Volume (ml),0.9800000000,3.4000000000,2.5000000000,0.9000000000
Enzyme activity (units/ml),276.8333333333,0.4450000000,67.7000000000,46.9142857143
Total enzyme (units),271.2966666667,1.5130000000,169.2500000000,42.2228571429
[Protein] (mg/ml),4.8810941638,1.0110867493,5.9082840801,8.0844196287
Total protein (mg),4.7834722805,3.4376949476,14.7707102002,7.2759776658
Specific activity (units/mg),56.7154256906,0.4401204944,11.4584876222,5.8030493058
Total protein relative to lysate (%),100.0000000000,71.8660994775,308.7863655125,152.1066129200
Enzyme yield relative to lysate (%),100.0000000000,0.5576920714,62.3855803610,15.5633527170
Fold purification relative to lysate,1.0000000000,0.0077601550,0.2020347636,0.1023187120


In [14]:
__logBase10of2 = 3.010299956639811952137388947244930267681898814621085413104274611e-1
def RoundToSigFigs_fp( x, sigfigs ):
    """
    Rounds the value(s) in x to the number of significant figures in sigfigs.
    Return value has the same type as x.

    Restrictions:
    sigfigs must be an integer type and store a positive value.
    x must be a real value.
    """
    if not ( type(sigfigs) is int or type(sigfigs) is long or
             isinstance(sigfigs, np.integer) ):
        raise TypeError( "RoundToSigFigs_fp: sigfigs must be an integer." )

    if sigfigs <= 0:
        raise ValueError( "RoundToSigFigs_fp: sigfigs must be positive." )

#     if not np.isreal( x.all() ):
#         raise TypeError( "RoundToSigFigs_fp: x must be real." )

    xsgn = np.sign(x)
    absx = xsgn * x
    mantissa, binaryExponent = np.frexp( absx )

    decimalExponent = __logBase10of2 * binaryExponent
    omag = np.floor(decimalExponent)

    mantissa *= 10.0**(decimalExponent - omag)

#     if mantissa < 1.0:
#         mantissa *= 10.0
#         omag -= 1.0

    return xsgn * np.around( mantissa, decimals=sigfigs - 1 ) * 10.0**omag

In [15]:
RoundToSigFigs_fp(EnzTab, 3)

,Lysate,Supernatent,SEC,DEAE
Volume (ml),0.98,3.40000,2.500,0.900
Enzyme activity (units/ml),277.00,0.44500,68.000,46.900
Total enzyme (units),271.00,1.51000,169.000,42.200
[Protein] (mg/ml),4.88,1.01000,5.910,8.100
Total protein (mg),4.78,3.44000,14.800,7.280
Specific activity (units/mg),56.70,0.44000,11.500,5.800
Total protein relative to lysate (%),100.00,72.00000,309.000,152.000
Enzyme yield relative to lysate (%),100.00,0.56000,62.400,15.600
Fold purification relative to lysate,1.00,0.00776,0.202,0.102


## Distance traveled vs molecular weight graph